In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Apparel_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32158956|R1KKOXHNI8MSXU|B01KL6O72Y|      24485154|Easy Tool Stainle...|         Apparel|          4|            0|          0|   N|                Y|★ THESE REALLY DO...|These Really Do W...| 2013-01-14|
|         US|    2714559|R26SP2OPDK4HT7|B01ID3ZS5W|     363128556|V28 Women Cowl Ne...|         Apparel|          5|    

In [16]:
df.columns

['marketplace',
 'customer_id',
 'review_id',
 'product_id',
 'product_parent',
 'product_title',
 'product_category',
 'star_rating',
 'helpful_votes',
 'total_votes',
 'vine',
 'verified_purchase',
 'review_headline',
 'review_body',
 'review_date']

In [17]:
test_product_type_df = df.select(["product_category"])
test_product_type_df.show()

+----------------+
|product_category|
+----------------+
|         Apparel|
|         Apparel|
|         Apparel|
|         Apparel|
|         Apparel|
|         Apparel|
|         Apparel|
|         Apparel|
|         Apparel|
|         Apparel|
|         Apparel|
|         Apparel|
|         Apparel|
|         Apparel|
|         Apparel|
|         Apparel|
|         Apparel|
|         Apparel|
|         Apparel|
|         Apparel|
+----------------+
only showing top 20 rows



In [18]:
from pyspark.sql.functions import to_date
vine_df = df.select(["review_id","star_rating","product_category","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+----------------+-------------+-----------+----+-----------------+
|     review_id|star_rating|product_category|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+----------------+-------------+-----------+----+-----------------+
|R1KKOXHNI8MSXU|          4|         Apparel|            0|          0|   N|                Y|
|R26SP2OPDK4HT7|          5|         Apparel|            1|          2|   N|                Y|
| RWQEDYAX373I1|          5|         Apparel|            0|          0|   N|                Y|
|R231YI7R4GPF6J|          5|         Apparel|            0|          0|   N|                Y|
|R3KO3W45DD0L1K|          5|         Apparel|            0|          0|   N|                Y|
|R1C4QH63NFL5NJ|          5|         Apparel|            0|          0|   N|                Y|
|R2GP65O1U9N7BP|          5|         Apparel|            0|          0|   N|                Y|
|R3O29CT5MQQ3XQ|          4|         Apparel|     

In [19]:
# Filtered for more than 20 total votes
vine_filtered_votes_df = vine_df.filter("total_votes>=20")
vine_filtered_votes_df.show()

+--------------+-----------+----------------+-------------+-----------+----+-----------------+
|     review_id|star_rating|product_category|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+----------------+-------------+-----------+----+-----------------+
|R35PT06NWP7LDP|          5|         Apparel|           30|         32|   N|                N|
|R2P76PJFUGXBHO|          1|         Apparel|            6|         45|   N|                N|
| RQW4AFOG9MR4Z|          5|         Apparel|           51|         52|   N|                N|
|R2SMUEBMGLAJQK|          5|         Apparel|           29|         36|   N|                Y|
|R1XVIZZALU5P6J|          5|         Apparel|          148|        150|   N|                Y|
|R11UFMG8M2488I|          4|         Apparel|           23|         26|   N|                N|
|R2OSHKSPXU68W5|          5|         Apparel|          136|        147|   N|                N|
|R3M02FSD3BLUPU|          5|         Apparel|     

In [20]:
# Filtered for more than 20 total votes and more than 50% helpful
vine_helpful_by_total_df = vine_filtered_votes_df.filter("(helpful_votes/total_votes)>=0.5")
vine_helpful_by_total_df.show()

+--------------+-----------+----------------+-------------+-----------+----+-----------------+
|     review_id|star_rating|product_category|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+----------------+-------------+-----------+----+-----------------+
|R35PT06NWP7LDP|          5|         Apparel|           30|         32|   N|                N|
| RQW4AFOG9MR4Z|          5|         Apparel|           51|         52|   N|                N|
|R2SMUEBMGLAJQK|          5|         Apparel|           29|         36|   N|                Y|
|R1XVIZZALU5P6J|          5|         Apparel|          148|        150|   N|                Y|
|R11UFMG8M2488I|          4|         Apparel|           23|         26|   N|                N|
|R2OSHKSPXU68W5|          5|         Apparel|          136|        147|   N|                N|
|R3M02FSD3BLUPU|          5|         Apparel|           19|         21|   N|                N|
|R1P0LEEJHH09L3|          5|         Apparel|     

In [21]:
# Filtered by reviews written as part of Vine program (paid)
vine_paid_helpful_df = vine_helpful_by_total_df.filter("vine == 'Y'")
vine_paid_helpful_df.show()

+--------------+-----------+----------------+-------------+-----------+----+-----------------+
|     review_id|star_rating|product_category|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+----------------+-------------+-----------+----+-----------------+
| R6U9701C3BGO6|          3|         Apparel|          139|        147|   Y|                N|
|R1XK3ALB45D7N4|          5|         Apparel|           33|         34|   Y|                N|
|R1IZCSTLX81D6C|          5|         Apparel|           31|         33|   Y|                N|
|R2C8NC8EQLH4JF|          3|         Apparel|           45|         48|   Y|                N|
|R1JJ1YOJMOML1P|          5|         Apparel|           18|         21|   Y|                N|
|R3TKG664L9MTXJ|          4|         Apparel|          164|        175|   Y|                N|
|R2E942L5EX73FP|          5|         Apparel|           29|         33|   Y|                N|
|R3CGRT9HYB3LI7|          5|         Apparel|     

In [22]:
# Filtered by unpaid review (not Vine)
vine_unpaid_helpful_df = vine_helpful_by_total_df.filter("vine == 'N'")
vine_unpaid_helpful_df.show()

+--------------+-----------+----------------+-------------+-----------+----+-----------------+
|     review_id|star_rating|product_category|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+----------------+-------------+-----------+----+-----------------+
|R35PT06NWP7LDP|          5|         Apparel|           30|         32|   N|                N|
| RQW4AFOG9MR4Z|          5|         Apparel|           51|         52|   N|                N|
|R2SMUEBMGLAJQK|          5|         Apparel|           29|         36|   N|                Y|
|R1XVIZZALU5P6J|          5|         Apparel|          148|        150|   N|                Y|
|R11UFMG8M2488I|          4|         Apparel|           23|         26|   N|                N|
|R2OSHKSPXU68W5|          5|         Apparel|          136|        147|   N|                N|
|R3M02FSD3BLUPU|          5|         Apparel|           19|         21|   N|                N|
|R1P0LEEJHH09L3|          5|         Apparel|     

In [24]:
# Total of all reviews

total_reviews_ct = vine_helpful_by_total_df.count()
total_reviews_ct


45421

In [25]:
# Total Vine reviews
total_paid_reviews_df = vine_helpful_by_total_df.filter("vine =='Y'")
total_paid_reviews_ct = total_paid_reviews_df.count()
total_paid_reviews_ct

33

In [26]:
# Total non-Vine reviews
total_unpaid_reviews_df = vine_helpful_by_total_df.filter("vine =='N'")
total_unpaid_reviews_ct = total_unpaid_reviews_df.count()
total_unpaid_reviews_ct

45388

In [27]:
# Total five-star reviews
total_five_star_reviews_df = vine_helpful_by_total_df.filter("star_rating == 5")
total_five_star_reviews_ct = total_five_star_reviews_df.count()
total_five_star_reviews_ct

23748

In [30]:
# Total five-star paid reviews
five_star_paid_reviews_df = total_paid_reviews_df.filter("star_rating == 5")
five_star_paid_reviews_ct = five_star_paid_reviews_df.count()
five_star_paid_reviews_ct

15

In [28]:
# Total five-star unpaid reviews
five_star_unpaid_reviews_df = total_unpaid_reviews_df.filter("star_rating < 5")
five_star_unpaid_reviews_ct = five_star_unpaid_reviews_df.count()
five_star_unpaid_reviews_ct

21655

In [31]:
# Five-star paid reviews as percent of total paid reviews
paid_five_star_per_total_paid = (five_star_paid_reviews_ct/total_paid_reviews_ct)*100
round(paid_five_star_per_total_paid, 3)

45.455

In [32]:
# Paid reviews as percent of total five-star reviews
paid_five_star_per_five_star_total = (five_star_paid_reviews_ct/total_five_star_reviews_ct)*100
round(paid_five_star_per_five_star_total, 3)

0.063

In [33]:
# Five-star unpaid reviews as percent of total unpaid reviews
unpaid_five_star_per_total_unpaid = (five_star_unpaid_reviews_ct/total_unpaid_reviews_ct)*100
round(unpaid_five_star_per_total_unpaid, 3)

47.711

In [34]:
# Five-star unpaid reviews as percent of total five-star reviews
unpaid_five_star_per_five_star_total = (five_star_unpaid_reviews_ct/total_five_star_reviews_ct)*100
round(unpaid_five_star_per_five_star_total, 3)

91.187